# Stage 3

- has **`location_raw`**: True
- vars one-hot encoded: False
- var label-encoded: True
- oversampled: False

In [4]:
import pickle

import humanize
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.externals.joblib import parallel_backend
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
X_train = pickle.load(open('./data/stage6-train.pkl', 'rb'))
y_train = X_train.pop('stop_outcome')
X_test = pickle.load(open('./data/stage6-test.pkl', 'rb'))
y_test = X_test.pop('stop_outcome')

print('Row counts:\n\ttrain: {}\n\ttest: {}'.format(X_train.shape[0], X_test.shape[0]))

Row counts:
	train: 222186
	test: 55547


In [3]:
# features = pickle.load(open('./data/stage3-l_encoded.pkl', 'rb'))
# labels = features.pop('stop_outcome')

In [4]:
# final_test_features = pickle.load(open('./data/final_test_set.pkl', 'rb'))
# final_test_outcomes = final_test_features.pop('stop_outcome')
# print('Final test set row count: {}'.format(final_test_features.shape[0]))

# X_train = features
# y_train = labels
# X_test = final_test_features
# y_test = final_test_outcomes

# Simple Linear Classifier

In [5]:
from sklearn import linear_model

clf_sgd = linear_model.SGDClassifier()
clf_sgd.fit(X_train, y_train)
clf_sgd.score(X_test, y_test)

/home/pato/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.68248510270581675

# VotingClassifier

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

clf1 = RandomForestClassifier(n_jobs=8, verbose=3, random_state=0)
clf2 = GaussianNB()
clf3 = DecisionTreeClassifier(random_state=0)
clf4 = GradientBoostingClassifier(verbose=3, random_state=0)

eclf = VotingClassifier(estimators=[('rf', clf1), ('gnb', clf2), ('dt', clf3), ('gb', clf4)],
                        voting='soft')
eclf = eclf.fit(X_train, y_train)
print('eclf score: {}'.format(eclf.score(X_test, y_test)))

building tree 4 of 10
building tree 5 of 10building tree 6 of 10building tree 1 of 10building tree 2 of 10building tree 7 of 10building tree 8 of 10building tree 3 of 10






building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.5s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.6s remaining:    0.2s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.8s finished


      Iter       Train Loss   Remaining Time 
         1      281283.1875            1.23m
         2      266377.5430            1.16m
         3      254461.9679            1.15m
         4      244809.7438            1.12m
         5      236933.1755            1.10m
         6      230389.0604            1.09m
         7      224984.9688            1.08m
         8      220253.9521            1.08m
         9      216417.4604            1.08m
        10      213112.6378            1.07m
        11      210269.1842            1.07m
        12      207924.1580            1.06m
        13      205713.8042            1.05m
        14      203931.8350            1.04m
        15      202204.6937            1.03m
        16      200696.2240            1.03m
        17      199482.6120            1.01m
        18      198379.0694            1.00m
        19      197438.5903           59.67s
        20      196566.2978           58.98s
        21      195826.7270           58.42s
        2

[Parallel(n_jobs=8)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


eclf score: 0.6808288476425369


In [7]:
eclf.estimators_

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=8,
             oob_score=False, random_state=0, verbose=3, warm_start=False),
 GaussianNB(priors=None),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=0,
             splitter='best'),
 GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impu

# GaussianNB

In [8]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print('{}'.format(gnb.score(X_test, y_test)))

0.6732496804507894


# DecisionTreeClassifier

In [9]:
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)
print('{}'.format(dtc.score(X_test, y_test)))

0.5399211478567699


# RandomForestClassifier

In [10]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
print('{}'.format(rfc.score(X_test, y_test)))

0.6320413343654923


# GradientBoostingClassifier

In [2]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X_train, y_train)
print('{}'.format(gbc.score(X_test, y_test)))

NameError: name 'GradientBoostingClassifier' is not defined

## GradientBoostingClassifier (Tuned)


In [ ]:
gbc_tuned = GradientBoostingClassifier(
    learning_rate=0.0983,
    max_depth=6,
    max_features=len(list(X_train.columns.values)),
    subsample=0.9,
    verbose=3,
    random_state=0,
)
gbc_tuned.fit(X_train, y_train)
print('{}'.format(gbc_tuned.score(X_test, y_test)))

# RandomizedSearchCV

In [ ]:
from sklearn.externals.joblib import parallel_backend
from sklearn.model_selection import RandomizedSearchCV, train_test_split


params = {
    'criterion': ['friedman_mse', 'mse', 'mae'],
#     'loss': ['deviance'],
    'learning_rate': [0.0983, 0.1],
    'max_depth': [5, 6, 7],
    'max_features': [None, len(list(X_train.columns.values))],
    'subsample': [0.5, 0.75, 0.9],
#     'verbose': [3],
#     'random_state': [0],

}

gbc = GradientBoostingClassifier(verbose=3, random_state=0)
clf = RandomizedSearchCV(gbc, params, scoring='accuracy', cv=5, verbose=3)

with parallel_backend('threading'):
    clf.fit(X_train, y_train)


In [ ]:
print(clf.score(X_test, y_test))

In [ ]:
clf.best_estimator_

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
n_estimators = 100
criterion='friedman_mse'
learning_rate=0.0983
max_depth=6
max_features=25
subsample=0.9
# gbc100 = GradientBoostingClassifier(verbose=3, random_state=0, criterion=criterion,learning_rate=learning_rate, max_depth=max_depth, max_features=max_features, subsample=subsample)
gbc100 = GradientBoostingClassifier(clf.best_params_)
gbc100.fit(X_train, y_train)
print('{}'.format(gbc100.score(X_test, y_test)))

# GradientBoostingClassifier

In [6]:
from sklearn.ensemble import GradientBoostingClassifier


gbc = GradientBoostingClassifier(random_state=0)
with parallel_backend('threading'):
    gbc.fit(X_train, y_train)
print('{}'.format(gbc.score(X_test, y_test)))

0.7080130340072371


## GradientBoostingClassifier (Tuned)


In [7]:
gbc_tuned = GradientBoostingClassifier(
    learning_rate=0.0983,
    max_depth=6,
    max_features=len(list(X_train.columns.values)),
    subsample=0.9,
    verbose=3,
    random_state=0,
)
with parallel_backend('threading'):
    gbc_tuned.fit(X_train, y_train)
print('{}'.format(gbc_tuned.score(X_test, y_test)))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      252173.4557        2021.5795            3.55m
         2      237869.7440        1554.8071            3.50m
         3      226537.7157        1284.3680            3.46m
         4      217022.3110        1025.0879            3.43m
         5      209507.3365         822.1559            3.43m
         6      203239.1616         697.5678            3.44m
         7      197916.5056         585.7970            3.43m
         8      193493.7245         471.2457            3.40m
         9      189747.8529         409.0788            3.37m
        10      186326.6897         348.7715            3.36m
        11      183742.5363         283.0833            3.32m
        12      181261.0246         244.2004            3.29m
        13      179342.0977         233.0752            3.28m
        14      177546.1736         189.8533            3.26m
        15      175727.0496         171.3121            3.25m
       